In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")

**A_N01_T**: Is the respondent male or female?

**B_Q01a**: Which of the qualifications on this card is the highest you have obtained?

**B_Q01b**: What was the area of study, emphasis or major for your highest level of qualification? If there was more than one, please choose the one you consider most important.

**F_Q04A**: How often ^DoesDid your ^JobLastjob usually involve persuading or influencing people?

**F_Q04B**: How often ^DoesDid your ^JobLastjob usually involve negotiating with people either inside or outside your firm or organisation?

**D_Q03**: In which sector of the economy do you work?

**D_Q06a**: How many people work for your employer at the place where you work? Would that be

**D_Q06b**: Over the last 12 months, has the number of people working at the place where you work 

**D_Q08b**: How many employees do you supervise or manage directly or indirectly? Would that be ...

**D_Q11d**: To what extent can you choose or change your working hours?

**D_Q11b**: To what extent can you choose or change how you do your work?

**D_Q12a**: Still talking about your current job:If applying today, what would be the usual qualifications, if any, that someone would need to GET this type of job?

**D_Q12b**: Thinking about whether this qualification is necessary for doing your job satisfactorily, which of the following statements would be most true?

**D_Q12c**: Supposing that someone with this level of qualification were applying today, how much related work experience would they need to GET this job? Would that be ...

**D_Q13c**: How often does your job involve keeping up to date with new products or services?

**D_Q14**: All things considered, how satisfied are you with your current job? Would you say you are ...

**F_Q02e**: How often ^DoesDid your ^JobLastjob usually involve advising people?

**F_Q02b**: How often ^DoesDid your ^JobLastjob usually involve instructing, training or teaching people, individually or in groups?

**F_Q05a**: Were you usually faced by relatively simple problems that ^TakeTook no more than 5 minutes to find a good solution?

**F_Q05b**: And how often ^AreWere you usually confronted with more complex problems that ^TakeTook at least 30 minutes to find a good solution?

**F_Q06b**: working physically for a long period?

**F_Q06c**: using skill or accuracy with your hands or fingers?

**G_Q04**: ^DoiDid you use a computer in your ^JobLastjob?

**G_Q05h**: participate in real-time discussions on the internet, for example online conferences, or chat groups?

**G_Q06**: What level of computer use ^IsWas needed to perform your ^JobLastjob?

**ISIC1C**: Industry classification of respondent's job at 1-digit level (ISIC rev 4), current job 

**ISCO1C**: Occupational classification of respondent's job at 1-digit level (ISCO 2008), current job

**YRSQUAL_T** : Derived variable on total years of schooling during lifetime - top coded at 24 (Trend-IALS/ALL)

**EARNMTHALLUS_C**: Monthly earnings

**D_Q18A_T**: (maybe can estimate EARNMTHALLUS_C with this)


In [2]:
df = pd.read_csv('Prgusap1_2017.csv', delimiter='|')

columns = ['ISIC1C', 'ISCO1C', 'YRSQUAL_T', 'A_N01_T','B_Q01A','B_Q01B','F_Q04A','F_Q04B','D_Q03','D_Q06B','D_Q08B','D_Q11D','D_Q11B','D_Q12A','D_Q12B','D_Q12C','D_Q13C','D_Q14','F_Q02E','F_Q02B','F_Q05A','F_Q05B','F_Q06B','F_Q06C','G_Q04','G_Q05H','G_Q06', 'EARNMTHALLUS_C']

print(df.shape)

df = df[df['D_Q04'] != '2']

df = df[columns]

df = df[df['EARNMTHALLUS_C'].str.replace('.', '').str.isnumeric()]

# discount self-employed people

print(df.shape)

df.head()

(3660, 1349)
(2005, 28)


,ISIC1C,ISCO1C,YRSQUAL_T,A_N01_T,B_Q01A,B_Q01B,F_Q04A,F_Q04B,D_Q03,D_Q06B,...,F_Q02E,F_Q02B,F_Q05A,F_Q05B,F_Q06B,F_Q06C,G_Q04,G_Q05H,G_Q06,EARNMTHALLUS_C
1,Q,2,19,1,13,4,2,1,1,3,...,5,4,5,4,4,5,1,1,2,23833.333333
8,C,9,14,1,11,7,4,2,1,1,...,5,3,5,3,4,5,1,1,1,6583.3333333
11,L,3,16,2,12,4,2,1,1,3,...,3,1,4,2,1,5,1,1,2,433.33333333
12,Q,5,12,2,7,N,5,4,2,1,...,5,2,5,2,5,5,2,V,V,379
13,P,1,18,1,13,4,3,4,3,3,...,4,2,5,4,1,5,1,2,2,4583.3333333


In [3]:
# Fixing the Vs

df['B_Q01B'] = df['B_Q01B'].replace('V', '0')
df['D_Q08B'] = df['D_Q08B'].replace('V', '0')
df['G_Q05H'] = df['G_Q05H'].replace('V', '1')
df['G_Q06'] = df['G_Q06'].replace('V', '0')

In [4]:
estimators = []
targets = []

# find all complete columns to estimate other columns
for column in df.columns:
    if df[df[column].astype(str).str.replace('.', '').str.isnumeric()].shape[0] < df.shape[0]:
        targets.append(column)
    else:
        estimators.append(column)
        
# drop gender from estimators
estimators.remove('A_N01_T')

#drop occupation from the targets
targets.remove('ISIC1C')

print(estimators)
print(targets)

['ISCO1C', 'B_Q01A', 'D_Q08B', 'D_Q11D', 'D_Q14', 'G_Q04', 'G_Q05H', 'EARNMTHALLUS_C']
['YRSQUAL_T', 'B_Q01B', 'F_Q04A', 'F_Q04B', 'D_Q03', 'D_Q06B', 'D_Q11B', 'D_Q12A', 'D_Q12B', 'D_Q12C', 'D_Q13C', 'F_Q02E', 'F_Q02B', 'F_Q05A', 'F_Q05B', 'F_Q06B', 'F_Q06C', 'G_Q06']


In [5]:
# make an estimating pipeline
def pipeline(target):
    global df
    global estimators
    # get integer values only
    test_df = df[df[target].astype(str).str.replace('.', '').str.isnumeric()]
    #make lin reg model or decision tree model
    y = test_df[target]
    X = test_df[estimators]
    clf = RandomForestClassifier(n_estimators=500, max_depth=5, random_state=0).fit(X, y)
    # target is df[target]
    # X is df[estimators]
    print(clf.score(X, y))
    #predict non numeric values for target and insert in somehow
    df['predictions'] = clf.predict(df[estimators])
    mask = df[target].astype(str).str.replace('.', '').str.isnumeric()
    df[target][~mask] = df['predictions'][~mask]
    # update estimators with new column
    estimators.append(target)
    return df, clf.score(X, y)

In [6]:
scores = []
for column in targets:
    df, score = pipeline(column)
    scores.append(score)
    
print(sum(scores)/len(scores))
    
df = df.drop(columns=['predictions'])

estimators = []
targets = []

# find all complete columns to estimate other columns
for column in df.columns:
    if df[df[column].astype(str).str.replace('.', '').str.isnumeric()].shape[0] < df.shape[0]:
        targets.append(column)
    else:
        estimators.append(column)

print(estimators)
print(targets)

# 0.6350690639840715

0.9164759725400458
0.42213642213642216
0.47954091816367267
0.5951073389915127
0.731
0.5855263157894737
0.4578132800798802
0.6483241620810405
0.6299376299376299
0.4844533600802407
0.4337831084457771
0.5553892215568862
0.48902195608782434
0.6087824351297405
0.4957563654518223
0.599600599101348
0.7869261477045908
0.7677322677322678
0.5937393056116763
['ISCO1C', 'YRSQUAL_T', 'A_N01_T', 'B_Q01A', 'B_Q01B', 'F_Q04A', 'F_Q04B', 'D_Q03', 'D_Q06B', 'D_Q08B', 'D_Q11D', 'D_Q11B', 'D_Q12A', 'D_Q12B', 'D_Q12C', 'D_Q13C', 'D_Q14', 'F_Q02E', 'F_Q02B', 'F_Q05A', 'F_Q05B', 'F_Q06B', 'F_Q06C', 'G_Q04', 'G_Q05H', 'G_Q06', 'EARNMTHALLUS_C']
['ISIC1C']


In [7]:
categories_list = ['9999', '9997']

mask = df['ISIC1C'].isin(categories_list)
test_df = df[~mask]
y = test_df['ISIC1C']
X = test_df.iloc[:, 1:]
clf = RandomForestClassifier(n_estimators=500, max_depth=5, random_state=0).fit(X, y)
df['predictions'] = clf.predict(df.iloc[:, 1:])
df['ISIC1C'][mask] = df['predictions'][mask]
df = df.drop(columns='predictions')

In [8]:
# convert to int or float

for column in df.columns:
    if column != 'ISIC1C':
        try:
            df[column] = df[column].astype('int8')
        except:
            df[column] = df[column].astype('float')
        
df.dtypes

ISIC1C             object
ISCO1C               int8
YRSQUAL_T            int8
A_N01_T              int8
B_Q01A               int8
B_Q01B               int8
F_Q04A               int8
F_Q04B               int8
D_Q03                int8
D_Q06B               int8
D_Q08B               int8
D_Q11D               int8
D_Q11B               int8
D_Q12A               int8
D_Q12B               int8
D_Q12C               int8
D_Q13C               int8
D_Q14                int8
F_Q02E               int8
F_Q02B               int8
F_Q05A               int8
F_Q05B               int8
F_Q06B               int8
F_Q06C               int8
G_Q04                int8
G_Q05H               int8
G_Q06                int8
EARNMTHALLUS_C    float64
dtype: object

In [9]:
df['EARNMTHALLUS_C']

1       23833.333333
8        6583.333333
11        433.333333
12        379.000000
13       4583.333333
            ...     
3649     4583.333333
3651     5858.333333
3653    11000.000000
3656      693.333333
3658      975.083333
Name: EARNMTHALLUS_C, Length: 2005, dtype: float64

In [10]:
column_names = {'ISIC1C':'industry', 'ISCO1C':'occupation', 'YRSQUAL_T':'yrs_qual', 'A_N01_T':'sex', 'B_Q01A':'highest_qual', 'B_Q01B':'area_of_study', 'F_Q04A':'influencing', 'F_Q04B':'negotiating', 'D_Q03':'sector', 'D_Q06B':'workforce_change', 'D_Q08B':'no_subordinates', 'D_Q11D':'choose_hours', 'D_Q11B':'choose_method', 'D_Q12A':'job_quals', 'D_Q12B':'qual_needed', 'D_Q12C':'experience_needed', 'D_Q13C':'keeping_current', 'D_Q14':'satisfaction', 'F_Q02E':'advising', 'F_Q02B':'instructing', 'F_Q05A':'problem_solving_quick', 'F_Q05B':'problem_solving_long', 'F_Q06B':'labour', 'F_Q06C':'manual_skill', 'G_Q04':'computer', 'G_Q05H':'group_meetings', 'G_Q06':'computer_level', 'EARNMTHALLUS_C':'monthly_salary'}

In [11]:
df = df.rename(columns=column_names)
df.head()

,industry,occupation,yrs_qual,sex,highest_qual,area_of_study,influencing,negotiating,sector,workforce_change,...,advising,instructing,problem_solving_quick,problem_solving_long,labour,manual_skill,computer,group_meetings,computer_level,monthly_salary
1,Q,2,19,1,13,4,2,1,1,3,...,5,4,5,4,4,5,1,1,2,23833.333333
8,C,9,14,1,11,7,4,2,1,1,...,5,3,5,3,4,5,1,1,1,6583.333333
11,L,3,16,2,12,4,2,1,1,3,...,3,1,4,2,1,5,1,1,2,433.333333
12,Q,5,12,2,7,4,5,4,2,1,...,5,2,5,2,5,5,2,1,0,379.000000
13,P,1,18,1,13,4,3,4,3,3,...,4,2,5,4,1,5,1,2,2,4583.333333


In [12]:
df.to_csv('wage_predict.csv', index=False)

In [13]:
df['industry'].unique()

array(['Q', 'C', 'L', 'P', 'G', 'I', 'E', 'J', 'N', 'A', 'M', 'O', 'K',
       'R', 'H', 'S', 'D', 'F', 'T', 'B', 'U'], dtype=object)